In [1]:
!pip install -q langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from langdetect import detect

In [3]:
# review pages (the complete dataset consists of more phones)
base_url = "https://www.gsmarena.com/"
phone_links = {
    "samsung_galaxy_a25": "samsung_galaxy_a25-reviews-12555p2.php",
    "google_pixel_8_pro": "google_pixel_8_pro-review-2629.php",
    "apple_iphone_15_pro_max": "apple_iphone_15_pro_max-review-2618.php",
    "huawei_nova_11_ultra": "huawei_nova_11_ultra-reviews-12230.php",
    "xiaomi_14_pro": "xiaomi_14_pro-reviews-12643p10.php",
}

In [4]:
phone_comments = {}

for phone, link in phone_links.items():
  url = f"{base_url}/{link}"
  # Sending a request to the website
  response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
  if not response.ok:
    print(f"Failed to get URL for {phone}")
    continue
  # Parsing the HTML content
  soup = BeautifulSoup(response.content, 'html.parser')
  # Extracting comments
  comments = soup.find_all("div", {"class": "user-thread"})
  phone_comments[phone] = comments

In [5]:
# Parse comments
parsed_comments = []
for phone, comments in phone_comments.items():
  for comment_html in comments:
    comment_raw = str(comment_html.find('p', {'class': 'uopin'}))
    comment = comment_raw.split("</span>")[1] if "</span>" in comment_raw else None
    item_data = {
        "Brand":phone.split("_")[0],
        "Model": " ".join(phone.split("_")[1:]),
        "Comment": comment,
        "Root": comment_html.find('p', {'class': 'uopin'}).get_text(strip=True),
    }
    parsed_comments.append(item_data)

In [6]:
df = pd.DataFrame(parsed_comments)

In [7]:
df['Language'] = df['Root'].apply(detect)

In [8]:
df.head()

,Brand,Model,Comment,Root,Language
0,samsung,galaxy a25,You don't make any sense. If you knew how to s...,"GSMA sponsored b Sas, 17 Feb 2024How come in t...",en
1,samsung,galaxy a25,Wrong there's a lot of midrange phones that us...,"Blind technology mas, 17 Feb 2024One of the re...",en
2,samsung,galaxy a25,None,How come in the review yoy point out a nwgativ...,en
3,samsung,galaxy a25,"<span class=""uinreply-msg uinreply-msg-single""...","Anonymous, 17 Feb 2024Excellent Exynos . All S...",en
4,samsung,galaxy a25,"<span class=""uinreply-msg uinreply-msg-single""...","Anonymous, 17 Feb 2024All mobiles must be go...",en
